# 卷积神经网络
## 从全连接层到卷积
卷积层 = 全连接层 + 平移不变性 + 局部性

## 图像卷积
二维交叉相关和二维卷积差不多，多了一个负号，在实际应用中没有区别。

卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出。

核矩阵和偏移是可学习的参数，核矩阵的大小是超参数。

说白了，就是利用卷积核将高维的输入映射为低维的输出，实现特征提取。

In [11]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  #@save 计算二维互相关运算
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [ ]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K) # 验证一下

In [ ]:
class Conv2D(nn.Module): # 继承nn.Module实现类
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [ ]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X # 边缘图像

In [ ]:
K = torch.tensor([[1.0, -1.0]]) # 卷积核
Y = corr2d(X, K)
Y

In [ ]:
corr2d(X.t(), K) # 经过转置后，该卷积核无法检测水平边缘。

In [ ]:
# 如何自动学习卷积核，而不是手动设置？
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)
# 核的形状可以设置，但具体数值还需进一步学习
# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8)) # 输入
Y = Y.reshape((1, 1, 6, 7)) # 输出
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

In [ ]:
conv2d.weight.data.reshape((1, 2))

## 卷积层里的填充和步幅
填充和步幅是卷积层的超参数

填充在输入周围添加额外的行/列，来控制输出形状的减少量

假设输入形状为$n_h\times n_w$，卷积核形状为$k_h\times k_w$，那么输出形状将是$(n_h-k_h+1) \times (n_w-k_w+1)$。

如果我们添加$p_h$行填充（大约一半在顶部，一半在底部）和$p_w$列填充（左侧大约一半，右侧一半），则输出形状将为

$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1)。$$

这意味着输出的高度和宽度将分别增加$p_h$和$p_w$。

通常取$p_h=k_h-1$和$p_w=k_w-1$，当$k_h$是奇数，在上下两侧填充$p_h/2$行。当$k_h$是偶数，则在上侧填充$\lceil p_h/2\rceil$行，在下侧填充$\lfloor p_h/2\rfloor$行。

卷积核的高度和宽度通常为奇数。

步幅是每次滑动核窗口时的行/列的步长，可以成倍的减少输出形状

通常，当垂直步幅为$s_h$、水平步幅为$s_w$时，输出形状为：

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

如果$p_h=k_h-1$和$p_w=k_w-1$，则输出形状为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。

如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为$(n_h/s_h) \times (n_w/s_w)$。

In [12]:
# 在所有侧边填充1个像素
import torch
from torch import nn

# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [13]:
# 填充不同的高度和宽度 2，1
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

NameError: name 'nn' is not defined

In [14]:
# 高度和宽度的步幅为2
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

NameError: name 'nn' is not defined

In [ ]:
# 高度和宽度的步幅为3、4，填充0和1
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape